In [38]:
# load json file 
import json
import os
import sys

import pandas as pd
import numpy as np

dataset_name = "IMDB"

paths = [f"output/runs/survey/{dataset_name}.json"]

datasets = []
for path in paths: 
    with open(path, 'r') as f:
        data = json.load(f)

    len(data)

    for entry in data:
        try :
            entry['lr'] = entry['parameters']['optimizer']['parameters']['lr']
        except KeyError:
            entry['lr'] = None

    df = pd.DataFrame(data)
    # remove dataset column 
    df = df.drop(columns=['dataset', 'parameters', 'sklearn.metrics.accuracy_score.test.original', 'sklearn.metrics.accuracy_score.forget.original', 'sklearn.metrics.accuracy_score.retain.original'])

    forget_accuracies = df['sklearn.metrics.accuracy_score.forget.unlearned']
    gold_accuracy = df['sklearn.metrics.accuracy_score.forget.unlearned'][0]
    df['forget_diff'] = np.abs(forget_accuracies - gold_accuracy)

    def nomus(f1, forget_score, l=0.5):
        return l * f1 + (1 - l) * (1-forget_score*2)
    
    df['Unlearning_Score'] = np.abs(df['UMIA'] - 0.5)

    df['nomus'] = nomus(df['sklearn.metrics.accuracy_score.test.unlearned'], df['Unlearning_Score'])

    # Remove relearning score 
    df = df.drop(columns=['RelearnTime'])

    # change column name RunTime to Time
    df = df.rename(columns={'RunTime': 'time'})

    

    
    datasets.append(df)

len(datasets)

1

In [39]:
datasets[0]

,unlearner,time,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.retain.unlearned,AUS,AIN,UMIA,lr,forget_diff,Unlearning_Score,nomus
0,GoldModel,2646.934836,0.889028,0.880271,0.925187,0.985543,1.000000,0.498516,NaN,0.000000,0.001484,0.943030
1,AdvancedNegGrad,2591.934002,0.494472,0.486670,0.502099,0.599101,20.451372,0.496274,0.00001,0.393602,0.003726,0.743510
2,BadTeaching,2297.964617,0.494472,0.486670,0.502099,0.597963,0.501247,0.500636,0.00001,0.393602,0.000636,0.746600
3,Finetuning,1273.329457,0.887881,0.925351,0.944118,0.960285,0.501247,0.522932,0.00001,0.045080,0.022932,0.921008
4,Finetuning,466.364223,0.892783,0.943286,0.935409,0.958944,0.002494,0.521418,0.00001,0.063015,0.021418,0.924974
5,eu_k,942.474342,0.892295,0.927664,0.935146,0.967659,0.004975,0.525538,0.00001,0.047392,0.025538,0.920610
6,Identity,0.000125,0.891669,0.925709,0.934937,0.960920,0.004975,0.519782,NaN,0.045437,0.019782,0.926053
7,NegGrad,69.554502,0.494526,0.512219,0.500717,0.591044,8.960199,0.500273,0.00001,0.368052,0.000273,0.746990
8,Scrub,1740.969058,0.828485,0.844086,0.866486,0.921912,1.000000,0.516752,0.00001,0.036185,0.016752,0.897490
9,SelectiveSynapticDampening,1379.937840,0.895319,0.929619,0.938587,0.970164,0.004975,0.518146,0.00001,0.049347,0.018146,0.929513


In [40]:
names = datasets[0].columns
names

Index(['unlearner', 'time', 'sklearn.metrics.accuracy_score.test.unlearned',
       'sklearn.metrics.accuracy_score.forget.unlearned',
       'sklearn.metrics.accuracy_score.retain.unlearned', 'AUS', 'AIN', 'UMIA',
       'lr', 'forget_diff', 'Unlearning_Score', 'nomus'],
      dtype='object')

In [41]:
evaluate_mean_std_for = ['sklearn.metrics.accuracy_score.test.unlearned', 'sklearn.metrics.accuracy_score.forget.unlearned', 'sklearn.metrics.accuracy_score.retain.unlearned', 'AUS', 'AIN', 'UMIA', 'forget_diff', 'Unlearning_Score', 'nomus']

new_df = pd.DataFrame(columns=names)

for name in names:
    if name in evaluate_mean_std_for:
        new_df[name] = np.mean([df[name] for df in datasets], axis=0)
        new_df[name + '_std'] = np.std([df[name] for df in datasets], axis=0)
    else:
        new_df[name] = datasets[0][name]

new_df

,unlearner,time,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.retain.unlearned,AUS,AIN,UMIA,lr,forget_diff,...,nomus,sklearn.metrics.accuracy_score.test.unlearned_std,sklearn.metrics.accuracy_score.forget.unlearned_std,sklearn.metrics.accuracy_score.retain.unlearned_std,AUS_std,AIN_std,UMIA_std,forget_diff_std,Unlearning_Score_std,nomus_std
0,GoldModel,2646.934836,0.889028,0.880271,0.925187,0.985543,1.000000,0.498516,NaN,0.000000,...,0.943030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AdvancedNegGrad,2591.934002,0.494472,0.486670,0.502099,0.599101,20.451372,0.496274,0.00001,0.393602,...,0.743510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BadTeaching,2297.964617,0.494472,0.486670,0.502099,0.597963,0.501247,0.500636,0.00001,0.393602,...,0.746600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Finetuning,1273.329457,0.887881,0.925351,0.944118,0.960285,0.501247,0.522932,0.00001,0.045080,...,0.921008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Finetuning,466.364223,0.892783,0.943286,0.935409,0.958944,0.002494,0.521418,0.00001,0.063015,...,0.924974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,eu_k,942.474342,0.892295,0.927664,0.935146,0.967659,0.004975,0.525538,0.00001,0.047392,...,0.920610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Identity,0.000125,0.891669,0.925709,0.934937,0.960920,0.004975,0.519782,NaN,0.045437,...,0.926053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,NegGrad,69.554502,0.494526,0.512219,0.500717,0.591044,8.960199,0.500273,0.00001,0.368052,...,0.746990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Scrub,1740.969058,0.828485,0.844086,0.866486,0.921912,1.000000,0.516752,0.00001,0.036185,...,0.897490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,SelectiveSynapticDampening,1379.937840,0.895319,0.929619,0.938587,0.970164,0.004975,0.518146,0.00001,0.049347,...,0.929513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
new_df['unlearner'][4] = "cfk"

/tmp/ipykernel_341485/974694434.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  new_df['unlearner'][4] = "cfk"
/tmp/ipykernel_341485/974694434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [43]:
# order columns name of the df 

new_df = new_df[['unlearner', 'lr', 'sklearn.metrics.accuracy_score.test.unlearned', 'sklearn.metrics.accuracy_score.test.unlearned_std', 'sklearn.metrics.accuracy_score.forget.unlearned', 'sklearn.metrics.accuracy_score.forget.unlearned_std', 'sklearn.metrics.accuracy_score.retain.unlearned', 'sklearn.metrics.accuracy_score.retain.unlearned_std', 'AUS', 'AUS_std', 'AIN', 'AIN_std', 'forget_diff', 'forget_diff_std', 'UMIA', 'UMIA_std', 'Unlearning_Score', 'Unlearning_Score_std', 'nomus', 'nomus_std', 'time']]

In [44]:
new_df['unlearner'][4] = "cfk"
new_df['unlearner'][9] = "ssd"

/tmp/ipykernel_341485/397938691.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  new_df['unlearner'][4] = "cfk"
/tmp/ipykernel_341485/397938691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [45]:
new_df

,unlearner,lr,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.test.unlearned_std,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.forget.unlearned_std,sklearn.metrics.accuracy_score.retain.unlearned,sklearn.metrics.accuracy_score.retain.unlearned_std,AUS,AUS_std,...,AIN_std,forget_diff,forget_diff_std,UMIA,UMIA_std,Unlearning_Score,Unlearning_Score_std,nomus,nomus_std,time
0,GoldModel,NaN,0.889028,0.0,0.880271,0.0,0.925187,0.0,0.985543,0.0,...,0.0,0.000000,0.0,0.498516,0.0,0.001484,0.0,0.943030,0.0,2646.934836
1,AdvancedNegGrad,0.00001,0.494472,0.0,0.486670,0.0,0.502099,0.0,0.599101,0.0,...,0.0,0.393602,0.0,0.496274,0.0,0.003726,0.0,0.743510,0.0,2591.934002
2,BadTeaching,0.00001,0.494472,0.0,0.486670,0.0,0.502099,0.0,0.597963,0.0,...,0.0,0.393602,0.0,0.500636,0.0,0.000636,0.0,0.746600,0.0,2297.964617
3,Finetuning,0.00001,0.887881,0.0,0.925351,0.0,0.944118,0.0,0.960285,0.0,...,0.0,0.045080,0.0,0.522932,0.0,0.022932,0.0,0.921008,0.0,1273.329457
4,cfk,0.00001,0.892783,0.0,0.943286,0.0,0.935409,0.0,0.958944,0.0,...,0.0,0.063015,0.0,0.521418,0.0,0.021418,0.0,0.924974,0.0,466.364223
5,eu_k,0.00001,0.892295,0.0,0.927664,0.0,0.935146,0.0,0.967659,0.0,...,0.0,0.047392,0.0,0.525538,0.0,0.025538,0.0,0.920610,0.0,942.474342
6,Identity,NaN,0.891669,0.0,0.925709,0.0,0.934937,0.0,0.960920,0.0,...,0.0,0.045437,0.0,0.519782,0.0,0.019782,0.0,0.926053,0.0,0.000125
7,NegGrad,0.00001,0.494526,0.0,0.512219,0.0,0.500717,0.0,0.591044,0.0,...,0.0,0.368052,0.0,0.500273,0.0,0.000273,0.0,0.746990,0.0,69.554502
8,Scrub,0.00001,0.828485,0.0,0.844086,0.0,0.866486,0.0,0.921912,0.0,...,0.0,0.036185,0.0,0.516752,0.0,0.016752,0.0,0.897490,0.0,1740.969058
9,ssd,0.00001,0.895319,0.0,0.929619,0.0,0.938587,0.0,0.970164,0.0,...,0.0,0.049347,0.0,0.518146,0.0,0.018146,0.0,0.929513,0.0,1379.937840


In [46]:
# # create a new dataset that, for each unlearner, has the one with the highest nomus score

# best = {}

# for i, row in new_df.iterrows():
#     if row['unlearner'] not in best:
#         best[row['unlearner']] = row
#     else:
#         if row['nomus'] > best[row['unlearner']]['nomus']:
#             best[row['unlearner']] = row

# best_df = pd.DataFrame(list(best.values()))
# best_df

In [47]:
# # read txt file with the times, where there is a line for each unlearner as "name time", for three times for the evaluation of mean and std 
# times = {}

# with open(f"output/runs/survey/time/{dataset_name}.txt", 'r') as f:
#     for line in f:
#         name, time = line.split(":")
#         if name not in times:
#             times[name] = []
#         times[name].append(float(time))

# times


In [48]:
# # for each entry, do the mean and std and add to the relative unlearner in the best_df

# # add time and time_std to the best_df
# best_df['time'] = 0
# best_df['time_std'] = 0

# for i, row in best_df.iterrows():
#     name = row['unlearner']
#     if name != 'Identity': 
#         best_df.loc[i, 'time'] = np.mean(times[name])
#         best_df.loc[i, 'time_std'] = np.std(times[name])
#     else:
#         best_df.loc[i, 'time'] = 0
#         best_df.loc[i, 'time_std'] = 0

# best_df

In [49]:
# evaluate speedup with respect to gold model 

gold_time = new_df[new_df['unlearner'] == 'GoldModel']['time'].values[0]
new_df['speedup'] = gold_time / new_df['time']
new_df

/tmp/ipykernel_341485/4039220614.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['speedup'] = gold_time / new_df['time']


,unlearner,lr,sklearn.metrics.accuracy_score.test.unlearned,sklearn.metrics.accuracy_score.test.unlearned_std,sklearn.metrics.accuracy_score.forget.unlearned,sklearn.metrics.accuracy_score.forget.unlearned_std,sklearn.metrics.accuracy_score.retain.unlearned,sklearn.metrics.accuracy_score.retain.unlearned_std,AUS,AUS_std,...,forget_diff,forget_diff_std,UMIA,UMIA_std,Unlearning_Score,Unlearning_Score_std,nomus,nomus_std,time,speedup
0,GoldModel,NaN,0.889028,0.0,0.880271,0.0,0.925187,0.0,0.985543,0.0,...,0.000000,0.0,0.498516,0.0,0.001484,0.0,0.943030,0.0,2646.934836,1.000000e+00
1,AdvancedNegGrad,0.00001,0.494472,0.0,0.486670,0.0,0.502099,0.0,0.599101,0.0,...,0.393602,0.0,0.496274,0.0,0.003726,0.0,0.743510,0.0,2591.934002,1.021220e+00
2,BadTeaching,0.00001,0.494472,0.0,0.486670,0.0,0.502099,0.0,0.597963,0.0,...,0.393602,0.0,0.500636,0.0,0.000636,0.0,0.746600,0.0,2297.964617,1.151861e+00
3,Finetuning,0.00001,0.887881,0.0,0.925351,0.0,0.944118,0.0,0.960285,0.0,...,0.045080,0.0,0.522932,0.0,0.022932,0.0,0.921008,0.0,1273.329457,2.078751e+00
4,cfk,0.00001,0.892783,0.0,0.943286,0.0,0.935409,0.0,0.958944,0.0,...,0.063015,0.0,0.521418,0.0,0.021418,0.0,0.924974,0.0,466.364223,5.675682e+00
5,eu_k,0.00001,0.892295,0.0,0.927664,0.0,0.935146,0.0,0.967659,0.0,...,0.047392,0.0,0.525538,0.0,0.025538,0.0,0.920610,0.0,942.474342,2.808495e+00
6,Identity,NaN,0.891669,0.0,0.925709,0.0,0.934937,0.0,0.960920,0.0,...,0.045437,0.0,0.519782,0.0,0.019782,0.0,0.926053,0.0,0.000125,2.114676e+07
7,NegGrad,0.00001,0.494526,0.0,0.512219,0.0,0.500717,0.0,0.591044,0.0,...,0.368052,0.0,0.500273,0.0,0.000273,0.0,0.746990,0.0,69.554502,3.805555e+01
8,Scrub,0.00001,0.828485,0.0,0.844086,0.0,0.866486,0.0,0.921912,0.0,...,0.036185,0.0,0.516752,0.0,0.016752,0.0,0.897490,0.0,1740.969058,1.520380e+00
9,ssd,0.00001,0.895319,0.0,0.929619,0.0,0.938587,0.0,0.970164,0.0,...,0.049347,0.0,0.518146,0.0,0.018146,0.0,0.929513,0.0,1379.937840,1.918155e+00


In [50]:
import pandas as pd

def highlight_best_and_second_best(df, column, ascending=True):
    filtered_df = df[~df['unlearner'].isin(['GoldModel', 'Identity'])]
    sorted_df = filtered_df.sort_values(by=column, ascending=ascending)
    best_value = sorted_df.iloc[0][column]
    second_best_value = sorted_df.iloc[1][column]
    return best_value, second_best_value

def format_value(value, best_value, second_best_value):
    if value == best_value:
        return f"\\textbf{{{value:.3f}}}"
    elif value == second_best_value:
        return f"\\underline{{{value:.3f}}}"
    else:
        return f"{value:.3f}"

def df_to_latex_table(df):
    # Define the LaTeX table header
    header = r"""\begin{table*}[]
\centering
\resizebox{\linewidth}{!}{
\begin{tabular}{l c c c c | c c | c c c | c | c}
\midrule\hline 

DS & Unlearner & Test Acc \up & Forget Acc \up & Retain Acc \up & AUS \up & AIN \down & Forget Diff \down & UMIA \down & Nomus \up & Time (Speedup) \down\\
\midrule\hline \\ 
"""

    # Get the best and second best values for each column
    best_test_acc, second_best_test_acc = highlight_best_and_second_best(df, 'sklearn.metrics.accuracy_score.test.unlearned', ascending=False)
    best_forget_acc, second_best_forget_acc = highlight_best_and_second_best(df, 'sklearn.metrics.accuracy_score.forget.unlearned', ascending=False)
    best_retain_acc, second_best_retain_acc = highlight_best_and_second_best(df, 'sklearn.metrics.accuracy_score.retain.unlearned', ascending=False)
    best_aus, second_best_aus = highlight_best_and_second_best(df, 'AUS', ascending=False)
    best_ain, second_best_ain = highlight_best_and_second_best(df, 'AIN', ascending=True)
    best_forget_diff, second_best_forget_diff = highlight_best_and_second_best(df, 'forget_diff', ascending=True)
    best_umia, second_best_umia = highlight_best_and_second_best(df, 'UMIA', ascending=True)
    best_nomus, second_best_nomus = highlight_best_and_second_best(df, 'nomus', ascending=False)
    best_time, second_best_time = highlight_best_and_second_best(df, 'time', ascending=True)

    # Format the data into LaTeX table rows
    rows = []
    for _, row in df.iterrows():
        # Convert values into "mean ± std" format
        test_acc = f"{format_value(row['sklearn.metrics.accuracy_score.test.unlearned'], best_test_acc, second_best_test_acc)} ± \scriptsize{{{row['sklearn.metrics.accuracy_score.test.unlearned_std']:.3f}}}"
        forget_acc = f"{format_value(row['sklearn.metrics.accuracy_score.forget.unlearned'], best_forget_acc, second_best_forget_acc)} ± \scriptsize{{{row['sklearn.metrics.accuracy_score.forget.unlearned_std']:.3f}}}"
        retain_acc = f"{format_value(row['sklearn.metrics.accuracy_score.retain.unlearned'], best_retain_acc, second_best_retain_acc)} ± \scriptsize{{{row['sklearn.metrics.accuracy_score.retain.unlearned_std']:.3f}}}"
        aus = f"{format_value(row['AUS'], best_aus, second_best_aus)} ± \scriptsize{{{row['AUS_std']:.3f}}}"
        ain = f"{format_value(row['AIN'], best_ain, second_best_ain)} ± \scriptsize{{{row['AIN_std']:.3f}}}"
        forget_diff = f"{format_value(row['forget_diff'], best_forget_diff, second_best_forget_diff)} ± \scriptsize{{{row['forget_diff_std']:.3f}}}"
        umia = f"{format_value(row['UMIA'], best_umia, second_best_umia)} ± \scriptsize{{{row['UMIA_std']:.3f}}}"
        nomus = f"{format_value(row['nomus'], best_nomus, second_best_nomus)} ± \scriptsize{{{row['nomus_std']:.3f}}}"
        time = f"{format_value(row['time'], best_time, second_best_time)}"
        speedup = f"{row['speedup']:.1f}"
        lr = row['lr']

        # Format the row
        rows.append(f"& {row['unlearner']} & {test_acc} & {forget_acc} & {retain_acc} & {aus} & {ain} & {forget_diff} & {umia} & {nomus} & {time} ({speedup}) \\\\")

    # Define the LaTeX table footer
    footer = r"""\midrule\hline 

\end{tabular}}
\caption{Experimental results of different unlearning methods.}
\label{tab:unlearning_results}
\end{table*}"""

    # Combine header, rows, and footer
    latex_table = "\n".join([header] + rows + [footer])

    return latex_table

new_df['unlearner'] = pd.Categorical(new_df['unlearner'], ['Finetuning', 'SuccessiveRandomLabels', 'cfk', 'eu_k', 'NegGrad', 'AdvancedNegGrad', 'unsir', 'BadTeaching', 'Scrub', 'FisherForgetting', 'ssd', 'salun', 'Identity', 'GoldModel'])

new_df = new_df.sort_values('unlearner')

latex_code = df_to_latex_table(new_df)

print(latex_code)  # Print LaTeX output

\begin{table*}[]
\centering
\resizebox{\linewidth}{!}{
\begin{tabular}{l c c c c | c c | c c c | c | c}
\midrule\hline 

DS & Unlearner & Test Acc \up & Forget Acc \up & Retain Acc \up & AUS \up & AIN \down & Forget Diff \down & UMIA \down & Nomus \up & Time (Speedup) \down\\
\midrule\hline \\ 

& Finetuning & 0.888 ± \scriptsize{0.000} & 0.925 ± \scriptsize{0.000} & \underline{0.944} ± \scriptsize{0.000} & 0.960 ± \scriptsize{0.000} & 0.501 ± \scriptsize{0.000} & 0.045 ± \scriptsize{0.000} & 0.523 ± \scriptsize{0.000} & 0.921 ± \scriptsize{0.000} & 1273.329 (2.1) \\
& SuccessiveRandomLabels & 0.890 ± \scriptsize{0.000} & 0.917 ± \scriptsize{0.000} & \textbf{0.946} ± \scriptsize{0.000} & 0.968 ± \scriptsize{0.000} & 1.000 ± \scriptsize{0.000} & \underline{0.037} ± \scriptsize{0.000} & 0.518 ± \scriptsize{0.000} & \underline{0.927} ± \scriptsize{0.000} & 1340.948 (2.0) \\
& cfk & \underline{0.893} ± \scriptsize{0.000} & \textbf{0.943} ± \scriptsize{0.000} & 0.935 ± \scriptsize{0.000} & 

/tmp/ipykernel_341485/1022938271.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['unlearner'] = pd.Categorical(new_df['unlearner'], ['Finetuning', 'SuccessiveRandomLabels', 'cfk', 'eu_k', 'NegGrad', 'AdvancedNegGrad', 'unsir', 'BadTeaching', 'Scrub', 'FisherForgetting', 'ssd', 'salun', 'Identity', 'GoldModel'])
